In [1]:
import re

import numpy as np
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [2]:
with open('../data/clean/misconduct_train.pkl', 'rb') as train_file:
    misconduct = pickle.load(train_file)

In [3]:
dummyized_cols = [
    'misconduct_type',
    'court_type',
    'enforcement_agency',
    'contracting_party'
]

indicator_features = [
    col for col in misconduct.columns
    if re.search(f'^({"|".join(dummyized_cols)})_', col)
]

In [4]:
misconduct_train, misconduct_validate = train_test_split(
    misconduct,
    test_size=0.2,
    random_state=59
)

In [5]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(
    misconduct_train.loc[:, indicator_features],
    misconduct_train['disposition_type']
)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [6]:
misconduct_validate.loc[:, indicator_features].iloc[9, :].shape

(107,)

In [7]:
predictions = knn_model.predict(
    misconduct_validate.loc[:, indicator_features].iloc[:, :]
)

offset_predictions = np.roll(predictions, 97)

In [8]:
# accuracy of guess with classes at preduction frequencies
(
    sum(offset_predictions == misconduct_validate['disposition_type']) /
    len(offset_predictions == misconduct_validate['disposition_type'])
)

0.2725

In [9]:
# accuracy
(
    sum(predictions == misconduct_validate['disposition_type']) /
    len(predictions == misconduct_validate['disposition_type'])
)

0.54